Question 1 - Extracting Tesla Stock Data Using yfinance - 2 Points

In [1]:
import yfinance as yf
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter("ignore")
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "browser"

tesla = yf.Ticker("TSLA")
stock_data = tesla.history(period="max")
stock_data.reset_index(inplace=True)
print("Stock Data for Tesla (first 5 rows):")
print(stock_data.head())

Stock Data for Tesla (first 5 rows):
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


Question 2 - Extracting Tesla Revenue Data Using Webscraping - 1 Points

In [2]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')
columns = ["Date", "Revenue"]
tesla_data = pd.DataFrame(columns=columns)
table = soup.find("table")
rows = table.find_all("tr")
for row in rows:
    cols = row.find_all("td")
    if len(cols) == 2:
        data = {
            "Date": cols[0].text.strip(),
            "Revenue": cols[1].text.strip()
        }
        tesla_data = pd.concat([tesla_data, pd.DataFrame([data])], ignore_index=True)
print(f"Total rows scraped: {len(tesla_data)}")
print(tesla_data.tail())  

Total rows scraped: 13
    Date Revenue
8   2013  $2,013
9   2012    $413
10  2011    $204
11  2010    $117
12  2009    $112


Question 3 - Extracting GameStop Stock Data Using yfinance - 2 Points

In [3]:
tesla = yf.Ticker("GME")
stock_data_gme = tesla.history(period="max")
stock_data_gme.reset_index(inplace=True)
print("Stock Data for GME (first 5 rows):")
print(stock_data_gme.head())

Stock Data for GME (first 5 rows):
                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620128  1.693350  1.603296  1.691666  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683251  1.687459  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


Question 4 - Extracting GameStop Revenue Data Using Webscraping - 1 Points

In [4]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')
columns = ["Date", "Revenue"]
gamestop_data = pd.DataFrame(columns=columns)
table = soup.find("table")
rows = table.find_all("tr") 
for row in rows:
    cols = row.find_all("td")
    if len(cols) == 2:
        data = {
            "Date": cols[0].text.strip(),
            "Revenue": cols[1].text.strip()
        }
        gamestop_data = pd.concat([gamestop_data, pd.DataFrame([data])], ignore_index=True)
print(f"Total rows scraped: {len(gamestop_data)}")
print(gamestop_data.tail())

Total rows scraped: 16
    Date Revenue
11  2009  $8,806
12  2008  $7,094
13  2007  $5,319
14  2006  $3,092
15  2005  $1,843


Question 5 - Tesla Stock and Revenue Dashboard - 2 Points

In [5]:
def make_graph(stock_df, revenue_df, title, start_date, end_date, color_stock, color_revenue):
    revenue_df["Date"] = pd.to_datetime(revenue_df["Date"])
    revenue_df["Revenue"] = revenue_df["Revenue"].str.replace(r"[\$,]", "", regex=True).astype(float)
    stock_df["Date"] = pd.to_datetime(stock_df["Date"])
    stock_df = stock_df[["Date", "Close"]]
    stock_filtered = stock_df[(stock_df["Date"] >= start_date) & (stock_df["Date"] <= end_date)]
    revenue_filtered = revenue_df[(revenue_df["Date"] >= start_date) & (revenue_df["Date"] <= end_date)]

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.15,
        subplot_titles=(f"{title} Stock Price", f"{title} Revenue")
    )

    fig.add_trace(
        go.Scatter(
            x=stock_filtered["Date"],
            y=stock_filtered["Close"],
            name="Stock Price",
            mode="lines",
            line=dict(color=color_stock)
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=revenue_filtered["Date"],
            y=revenue_filtered["Revenue"],
            name="Revenue",
            mode="lines+markers",
            line=dict(color=color_revenue)
        ),
        row=2, col=1
    )

    fig.update_layout(
        height=800,
        title_text=f"{title} Stock and Revenue Dashboard",
        xaxis=dict(range=[start_date, end_date]),
        xaxis2=dict(range=[start_date, end_date]),
        xaxis_rangeslider_visible=True,
        showlegend=False
    )

    fig.update_yaxes(title_text="Stock Price ($)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($)", row=2, col=1)

    fig.show()

make_graph(stock_data, tesla_data, "Tesla", "2009-01-01", "2021-12-31", "blue", "green")

Question 6 - GameStop Stock and Revenue Dashboard - 2 Points

In [6]:
make_graph(stock_data_gme, gamestop_data, "GameStop", "2005-01-01", "2021-12-31", "darkred", "darkgreen")
